# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [2]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [4]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 09e3f9f6


In [5]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [6]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [7]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [8]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [9]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/abirvabdeb/git/AIE9/07_Deep_Agents/workspace


In [10]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] comprehensive_sleep_improvement_guide.md (18820 bytes)
[FILE] comprehensive_stress_management_guide.md (9312 bytes)
[FILE] personal_sleep_improvement_plan.md (6388 bytes)
[DIR] research
[DIR] workspace


In [11]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [12]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [13]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/abirvabdeb/git/AIE9/07_Deep_Agents/workspace


In [14]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Sleep Improvement Plan is Complete! 🌙

I've created a comprehensive, evidence-based sleep improvement plan tailored specifically to your three main challenges. The plan is now saved in `/personalized_sleep_improvement_plan.md` for easy reference.

### Key Highlights of Your Plan:

**🎯 Targeted Solutions:**
- **Inconsistent bedtimes**: Gradual 15-minute adjustments toward 10:30 PM target
- **Phone in bed**: Complete bedroom phone elimination + digital sunset protocol
- **Morning fatigue**: Fixed 7:00 AM wake time + immediate morning light exposure

**📅 3-Phase Approach:**
1. **Weeks 1-2**: Foundation building (wake time consistency, basic phone boundaries)
2. **Weeks 3-4**: Sleep schedule optimization (bedtime progression, wind-down routines)
3. **Weeks 5-8**: Advanced fine-tuning (breathing techniques, environment optimization)

**🚀 Tonight's Quick-Start Actions:**
1. Set phone to airplane mode at 9:00 PM
2. Remove phone from bedroom entirely
3. Kee

In [15]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep improvement strategies (completed)
✅ [todo_5] Research circadian rhythm regulation and sleep schedule consistency (completed)
✅ [todo_7] Save plan to file (completed)
✅ [todo_6] Investigate blue light exposure and screen time effects on sleep (completed)
✅ [todo_8] Research sleep environment optimization techniques (completed)
✅ [todo_10] Study pre-sleep routines and behavioral interventions (completed)
✅ [todo_12] Research morning fatigue causes and morning routine adjustments (completed)
✅ [todo_14] Compile gradual implementation strategies (completed)
✅ [todo_16] Create structured summary with actionable recommendations (completed)


Workspace contents:
  [FILE] comprehensive_sleep_improvement_guide.md (18820 bytes)
  [FILE] comprehensive_stress_management_guide.md (9312 bytes)
  [FILE] evidence_based_sleep_solutions.md (7515 bytes)
  [FILE] personal_sleep_improvement_p

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
- a simple task would be much slower (more planning slows down simple tasks)
- when there is an issue, the todo should probably be updated
- subagents are used to get new context windows
- granularity is all about context windows; too granular will fill out too many tokens (context window will overflow/overplan; too many steps and context window will overflow)

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
- semantic retrieval / RAG for a large document because this is new knowledge we are adding
- use metrics can be stored in long-term storage
- skills/capabilities of an agent usually go into files (for proper context management)
  - you let the LLM decide when to utilize skills for proper context management
  - prompts are context that is forced upon the LLM; they do not decide what goes into the system prompt, but they do decide when a skill has proper context for the task at hand
- sensitive information probably should not be in files

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [16]:
### YOUR CODE HERE ###
from pathlib import Path
from langchain_core.tools import tool
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide


# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")
# --- Tool: read wellness guide from data folder ---
WELLNESS_GUIDE_PATH = Path("data/HealthWellnessGuide.txt")

@tool
def read_wellness_guide() -> str:
    """Read the Health & Wellness Guide document from the data folder.

    Use this to research wellness topics (e.g., stress management, sleep, nutrition).
    Returns the full text of the guide.
    """
    path = WELLNESS_GUIDE_PATH
    if not path.exists():
        return f"File not found: {path}. Ensure you're running from 07_Deep_Agents and the data folder exists."
    return path.read_text()


# --- Backend and tools ---
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True,
)

custom_tools = [
    write_todos,
    update_todo,
    list_todos,
    read_wellness_guide,
]

SYSTEM_PROMPT = """You are a Research Agent that produces structured wellness reports.

When given a research task:
1. Create a todo list (e.g., read guide, extract content, structure report, write file).
2. Use read_wellness_guide to read the Health & Wellness Guide from the data folder.
3. Save your findings to a structured markdown file in the workspace (use write_file).
4. Update todo status (update_todo) as you complete each step.

File paths: You are already inside the workspace. When using write_file or edit_file, use paths relative to the workspace root only, e.g. "stress_guide.md" or "research/stress_guide.md". Do NOT include "workspace" in the path.

Be evidence-based and thorough. Output clear sections and at least 5 strategies when asked."""

# Step 3: Create the agent with a research-focused system prompt
research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,
    system_prompt=SYSTEM_PROMPT,
)


# Step 4: Test with the stress management research task
# clear TODO_STORE from past research tasks
TODO_STORE.clear()
result = research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."
    }]
})
print(result["messages"][-1].content)

Perfect! I've successfully completed your comprehensive stress management guide. Here's what I've created for you:

## Summary of the Comprehensive Stress Management Guide

I've researched and compiled a thorough, evidence-based stress management guide saved as `stress_management_guide.md` in your workspace. The guide includes:

### **7 Evidence-Based Stress Management Strategies:**

1. **Deep Breathing Techniques** - Box breathing method with scientific backing
2. **Progressive Muscle Relaxation** - Systematic tension-release technique
3. **Mindfulness Meditation** - Present-moment awareness practices with neuroimaging evidence
4. **Regular Physical Exercise** - Multiple exercise options with endorphin research
5. **Social Connection & Support** - Relationship-building for stress buffering
6. **Sleep Optimization** - Evidence-based sleep hygiene practices
7. **Time Management & Boundary Setting** - Practical organizational strategies

### **Key Features of the Guide:**
- **Scientific 

- looks like the research agent was able to utilize the read_wellness_guide tool to research stress management techniques

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [17]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [18]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [19]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what was accomplished:

## ✅ **Project Complete - All Tasks Finished**

### **📋 What Was Delivered:**

1. **🔬 Scientific Research** - In-depth research on:
   - Circadian rhythm science and morning light exposure
   - Exercise research (20-30 minute morning workouts)
   - Nutrition science for sustained energy
   - Mindset and neuroscience research on morning practices
   - Sleep science connections

2. **📖 Comprehensive Guide** - A complete 7-section guide including:
   - **Engaging introduction** that motivates readers
   - **Science section** with research-backed explanations
   - **Three-pillar framework**: Exercise, Nutrition, and Mindset
   - **Step-by-step implementation** with 4 progressive phases
   - **Timing options**: Express (15-20 min), Balanced (30-40 min), Full (60-90 min)
   - **Troubleshooting** for 8 common challenges
   - **Quick reference** wi

In [20]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines and energy (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Save the guide as a markdown file (completed)

Generated files in workspace:
  [FILE] comprehensive_morning_energy_guide.md (14180 bytes)
  [FILE] comprehensive_sleep_improvement_guide.md (18820 bytes)
  [FILE] comprehensive_stress_management_guide.md (9312 bytes)
  [FILE] evidence_based_sleep_solutions.md (7515 bytes)
  [FILE] morning_energy_guide.md (40031 bytes)
  [FILE] morning_routine_energy_guide.md (875 bytes)
  [FILE] personal_sleep_improvement_plan.md (6388 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5724 bytes)
  [DIR] research/
  [FILE] stress_management_guide.md (11693 bytes)
  [DIR] workspace/


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [21]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [22]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [23]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [24]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! Great to see you again. Based on your profile, I remember that your primary goal is to improve energy levels and get better sleep, and you prefer morning workouts. Given your mild anxiety, I'll recommend exercises that can help boost energy while promoting relaxation.

Here's a personalized exercise routine that aligns with your goals:

## **Morning Energy & Sleep Routine for Alex**

### **Week Structure (4-5 days/week):**

**Monday, Wednesday, Friday - Energizing Cardio + Strength**
- 5-minute gentle warm-up (walking or light stretching)
- 20-25 minutes moderate cardio (brisk walking, cycling, or swimming)
- 15 minutes bodyweight strength training:
  - Squats, push-ups (modified as needed), planks, lunges
- 5-minute cool-down with deep breathing

**Tuesday, Thursday - Mind-Body Focus**
- 20-30 minutes yoga or tai chi (excellent for anxiety management)
- Focus on poses that energize (sun salutations) and calm (child's pose, legs up the wall)
- End with 5 minute

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [25]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [26]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [27]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [28]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [29]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Wellness Assessment Summary

I've completed a comprehensive wellness assessment and created a personalized plan for you. Here are the key findings:

### Your Wellness Strengths:
- **Healthy dietary foundation** with your vegetarian lifestyle
- **Health awareness** - you're being proactive about your wellness
- **No major health barriers** to work around
- **Clear, specific goals** (15-pound weight loss)

### Priority Focus Areas:
1. **Sleep Quality** (Primary) - This is your #1 priority because poor sleep directly sabotages weight loss and energy levels
2. **Movement Integration** (Secondary) - Combat your sedentary work life with strategic movement
3. **Weight Management** (Tertiary) - Build sustainable habits for your 15-pound goal

### Your Action Plan:

**Start Today:**
- Set up a proper sleep environment (no screens 1 hour before bed, consistent sleep schedule)
- Take 2-minute movement breaks every hour at work
- Boost hydration with morning water and desk 

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [31]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents 0.0.19


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [32]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [33]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [34]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! I've created your comprehensive 2-week wellness program with all the components you requested. Here's what I've delivered:

## 🎯 **Your Complete Wellness Program Files:**

1. **`/alex_2week_exercise_plan.txt`** - Complete exercise program with:
   - 3 workouts per week (Monday/Wednesday/Friday)
   - 30-minute sessions progressing from Week 1 to Week 2
   - Mix of cardio, strength training, and mindful movement
   - Modifications for different fitness levels
   - Focus on building consistency and reducing anxiety

2. **`/alex_2week_vegetarian_meal_plan.txt`** - Comprehensive nutrition plan with:
   - 14 days of complete vegetarian meals
   - Shopping lists for both weeks
   - Energy-boosting and anxiety-supporting foods
   - Simple, practical recipes
   - Meal prep tips and protein sources

3. **`/alex_2week_stress_sleep_plan.txt`** - Stress and sleep optimization with:
   - Daily stress management techniques
   - Evening wind-down routines
   - Workpla

In [35]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create 2-week exercise program (completed)
✅ [todo_3] Develop vegetarian meal plan (completed)
✅ [todo_5] Design stress & sleep optimization plan (completed)
✅ [todo_7] Create integrated wellness program (completed)
✅ [todo_9] Save all plans to files (completed)

GENERATED FILES
  [FILE] alex_2week_exercise_plan.txt (4276 bytes)
  [FILE] alex_2week_integrated_wellness_program.txt (7080 bytes)
  [FILE] alex_2week_stress_sleep_plan.txt (5913 bytes)
  [FILE] alex_2week_vegetarian_meal_plan.txt (6328 bytes)
  [FILE] comprehensive_morning_energy_guide.md (14180 bytes)
  [FILE] comprehensive_sleep_improvement_guide.md (18820 bytes)
  [FILE] comprehensive_stress_management_guide.md (9312 bytes)
  [FILE] evidence_based_sleep_solutions.md (7515 bytes)
  [FILE] morning_energy_guide.md (40031 bytes)
  [FILE] morning_routine_energy_guide.md (875 bytes)
  [FILE] personal_sleep_improvement_plan.md (6388 bytes)
  [FILE] personalized_sleep_improvement_plan.md (5724 bytes)


In [36]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of comprehensive_morning_energy_guide.md:
# 🌅 The Complete Morning Energy Guide

*Transform Your Days with Science-Based Morning Routines*

---

## Table of Contents

1. [Introduction: Your Energy Revolution Starts at Dawn](#introduction)
2. [The Science Behind Morning Energy](#the-science)
3. [The Three-Pillar Framework](#the-framework)
4. [Step-by-Step Implementation Guide](#implementation)
5. [Timing and Customization](#timing)
6. [Common Challenges and Solutions](#troubleshooting)
7. [Quick Reference Guide](#quick-reference)

---

## Introduction: Your Energy Revolution Starts at Dawn {#introduction}

Imagine waking up naturally, feeling refreshed and ready to tackle whatever the day brings. Picture yourself moving through your morning with purpose, nourishing your body with intention, and setting your mind for success—all before most people have even opened their eyes.

This isn't just a dream. It's the reality for millions who have discovered the transformative power of

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
- subagents should share functionality like file systems (for skills), or searching the web (for research, if they are all research subagents)
- which model depends on the task difficulty and cost considerations
- subagent should be doing a specific "type" of task that doesn't necessarily need all the context that a supervisor agent would need for planning
  - IE: a subagent should be broad enough to be useful but narrow enough to be specific)
  - subagents usually have narrower focused tasks, so they can use smaller/faster models
  - supervisor need more planning so they need larger/more sophisticated models (usually)

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
- safety guardrails like under-aged kids utilizing the app
  - need guardrails based on age, medical disclaimers b/c this is not replacement for a doctor's advice
  - guardrails can be in the system prompts or even in skills
  - need to check compliance with medical industry laws
  - should be able to handle emergencies (or tell person to call 911 if emergency)
- persistent storage is best for storing user preferences (but should have guardrails for not storing sensitive information)
- multi-user support -> different namespaces on different persistent storage
- LangSmith -> need to be able to traceback for observability and decisions made by the Wellness App
- are medications something that can be stored in persistent storage? Is that considered sensitive data that should not be stored in persistent data?

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [38]:
### YOUR CODE HERE ###
from pathlib import Path
from langchain_core.tools import tool
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Step 1: Define your subagent configurations
plan_specialist = {
    "name": "plan-specialist",
    "description": "Use this agent to design personalized 30-day wellness plans (exercise, nutrition, sleep, habits).",
    "system_prompt": """You are a wellness plan specialist. Your job is to:
1. Create structured, day-by-day or week-by-week plans
2. Personalize based on user goals and constraints
3. Keep plans realistic and progressive

Output clear, actionable plans. Consider fitness level, time, and preferences.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

progress_specialist = {
    "name": "progress-specialist",
    "description": "Use this agent to review daily check-ins, track progress, and write weekly summary reports.",
    "system_prompt": """You are a progress and reporting specialist. Your job is to:
1. Summarize daily check-in notes into weekly reports
2. Identify patterns and suggest adaptations
3. Write encouraging, actionable weekly summaries

When writing a weekly summary, you may receive prior weekly summaries and daily check-ins from the coordinator; use them to identify trends and avoid repeating the same advice.

Be concise and focus on what changed and what to adjust.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}


# Step 2: Create any additional tools you need
CHECKINS_DIR = WORKSPACE / "checkins"
SUMMARIES_DIR = WORKSPACE / "summaries"
CHECKINS_DIR.mkdir(parents=True, exist_ok=True)
SUMMARIES_DIR.mkdir(parents=True, exist_ok=True)

@tool
def save_daily_checkin(day: int, notes: str) -> str:
    """Save daily check-in notes for the 30-day challenge (context management).
    Args:
        day: Day number (1-30)
        notes: User's check-in notes (mood, completion, feedback)
    Returns:
        Confirmation message
    """
    path = CHECKINS_DIR / f"day_{day:02d}.md"
    path.write_text(notes)
    return f"Saved check-in for day {day} to {path.name}"

@tool
def save_weekly_summary(week: int, content: str) -> str:
    """Save a weekly summary report for the 30-day challenge.
    Args:
        week: Week number (1-4)
        content: Summary text (progress, adaptations, next steps)
    Returns:
        Confirmation message
    """
    path = SUMMARIES_DIR / f"week_{week}.md"
    path.write_text(content)
    return f"Saved weekly summary for week {week} to {path.name}"

@tool
def read_daily_checkin(day: int) -> str:
    """Read daily check-in notes for a given day (for adaptation).
    Args:
        day: Day number (1-30)
    Returns:
        Check-in content or a message if not found
    """
    path = CHECKINS_DIR / f"day_{day:02d}.md"
    if not path.exists():
        return f"No check-in found for day {day}"
    return path.read_text()

@tool
def read_weekly_summary(week: int) -> str:
    """Read a weekly summary report for the 30-day challenge.
    Args:
        week: Week number (1-4)
    Returns:
        Summary content or a message if not found
    """
    path = SUMMARIES_DIR / f"week_{week}.md"
    if not path.exists():
        return f"No summary found for week {week}"
    return path.read_text()

# Step 3: Build the main coordinator agent
workspace_path = Path("workspace").resolve()
workspace_path.mkdir(exist_ok=True)
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True,
)

SYSTEM_PROMPT = """You are a 30-Day Wellness Challenge Coach. Your role is to:

1. **Planning**: Create and maintain a 30-day todo list (use write_todos for the plan phases).
2. **Context**: Store daily check-ins with save_daily_checkin and weekly reports with save_weekly_summary. Use read_daily_checkin and read_weekly_summary when you need past context—e.g. before writing a new weekly summary, when adapting recommendations, or when the user asks about their progress.
3. **Memory**: Use get_user_profile and save_user_preference to remember preferences and adapt.
4. **Subagents**: Delegate to plan-specialist for building the 30-day plan; delegate to progress-specialist for weekly summaries and adaptation advice.

Workflow:
- On onboarding: get user profile, create 30-day todos, ask plan-specialist to generate a personalized plan and save it to a file.
- On daily check-in: save_daily_checkin(day, notes), then update_todo as needed; adapt recommendations using profile and check-in history.
- Each week: Use read_weekly_summary for previous weeks (if any) and read_daily_checkin for that week’s days to give the progress-specialist full context; then ask progress-specialist to produce the weekly summary and save it with save_weekly_summary.

Always personalize and adapt based on feedback and stored preferences."""

wellness_challenge_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        write_todos,
        update_todo,
        list_todos,
        save_daily_checkin,
        save_weekly_summary,
        read_daily_checkin,
        read_weekly_summary,
        get_user_profile,
        save_user_preference,
    ],
    backend=filesystem_backend,
    subagents=[plan_specialist, progress_specialist],
    system_prompt=SYSTEM_PROMPT,
)

# clear TODO list from previous demos
TODO_STORE.clear()

In [39]:
# Step 4: Test with a user creating their 30-day challenge
result = wellness_challenge_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """My user_id is user_alex. I want to start the 30-day wellness challenge.

Please:
1. Create a personalized 30-day plan for me (use my profile) and save it to a file.
2. Set up todos so we can track the 30-day challenge.
3. Tell me what to do for day 1."""
    }]
})

In [42]:
print("Wellness Coach Agent response:")
print(result["messages"][-1].content)

Wellness Coach Agent response:
Perfect! Now I have everything set up for your wellness challenge. Here's what's ready for you:

## 🎉 Your 30-Day Wellness Challenge is Ready!

**What's been set up:**
1. ✅ **Personalized Plan Created** - A comprehensive 30-day plan tailored to your profile (vegetarian, mild anxiety, morning exercise preference, goals of improving energy and sleep)
2. ✅ **Todo Tracking System** - 7 major todos to track your weekly progress and daily activities
3. ✅ **Plan File Saved** - Your complete plan is saved and ready for reference

## 📅 Day 1 (Monday) - Foundation Day

Based on your personalized plan, here's what to focus on today:

### 🏃‍♂️ **Exercise** (Morning - as you prefer!)
- **20 minutes brisk walking** + **10 minutes bodyweight exercises**
- Bodyweight exercises: squats, push-ups, planks
- Start gentle since it's Day 1!

### 🥗 **Nutrition** (Vegetarian-friendly)
- **Breakfast**: Oatmeal with fruits
- **Lunch**: Grilled vegetarian protein salad 
- **Dinner*

In [44]:
# Step 5: Simulate a daily check-in and adaptation
result2 = wellness_challenge_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Daily check-in for day 1: I did the morning exercise, and was able to get all vegan nutritious meals during the day, but I was late with going to bed on-time.
        Can you save this check-in and suggest what to do for day 2, please?"""
    }]
})

print("\nWellness Coach Agent response (after daily check-in):")
print(result2["messages"][-1].content)


Wellness Coach Agent response (after daily check-in):
Great job on completing day 1! You had some excellent wins - completing your morning exercise and sticking to your vegan nutrition plan. That shows great commitment to your wellness goals.

## For Day 2, here are my suggestions:

**🎯 Focus Area: Sleep Timing**
Since you were late going to bed, let's make sleep your priority focus for day 2:

- **Set a wind-down alarm**: Set an alarm 1 hour before your target bedtime to start your evening routine
- **Create a bedtime ritual**: 30 minutes before bed, try dimming lights, putting away devices, and doing something calming (reading, gentle stretching, or meditation)
- **Consider your evening meal timing**: Make sure you finish eating at least 2-3 hours before bedtime

**💪 Build on Your Strengths:**
- **Morning exercise**: You nailed this yesterday! Keep the same routine that worked
- **Vegan nutrition**: You did great with this - continue planning your meals ahead of time

**📝 Day 2 Acti

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)